In [25]:
import pandas as pd
import altair as alt
import os

In [26]:
healthcare_dir = 'Healthcare'
tech_dir = 'Tech'

# Dictionary to hold dataframes
healthcare = {}
tech = {}

In [27]:
# Add fundamental data to pandas dataframe, append to dictionary based on industry
def load_spreadsheets_from_folder(folder_path, dataframes_dict):
    for filename in os.listdir(folder_path):
        if filename.endswith('_fundamentals.csv'):
            ticker = filename.split('_')[0]  # Assuming the filename format is "TICKER_fundamentals.csv"
            file_path = os.path.join(folder_path, filename)
            dataframes_dict[ticker] = pd.read_csv(file_path)

load_spreadsheets_from_folder(healthcare_dir, healthcare)
load_spreadsheets_from_folder(tech_dir, tech)

In [28]:
for ticker, df in tech.items():
    df['Industry'] = 'Tech'  # Add a new column with value 'Tech'

for ticker, df in healthcare.items():
    df['Industry'] = 'Healthcare'  # Add a new column with value 'Healthcare'

In [29]:
# Group by quarterly and get average of columns for each quarter
def aggregate_by_q(df, date_column='Date'):
    # Convert date column to datetime
    df[date_column] = pd.to_datetime(df[date_column])
    # Set the date column as the index
    df.set_index(date_column, inplace=True)
    numeric_cols = df.select_dtypes(include=['number'])
    # Resample by month and calculate the mean
    df_q = numeric_cols.resample('QE').mean()
    
    non_numeric_cols = df.select_dtypes(exclude=['number'])
    for col in non_numeric_cols.columns:
        # Take the first value encountered in each quarter for the non-numeric column
        first_non_numeric = non_numeric_cols[col].resample('QE').first()
        # Add this non-numeric column back to the df_q DataFrame
        df_q[col] = first_non_numeric
    
    return df_q

for ticker, df in healthcare.items():
    healthcare[ticker] = aggregate_by_q(df)
    healthcare[ticker]['Ticker'] = ticker

for ticker, df in tech.items():
    tech[ticker] = aggregate_by_q(df)
    tech[ticker]['Ticker'] = ticker

In [30]:
# Function to concatenate all DataFrames in a given dictionary
def reset_index_if_needed(df):
    if 'Date' not in df.columns:
        df = df.reset_index()
    return df

# Use the helper function to prepare DataFrames
healthcare_dfs = [reset_index_if_needed(df) for df in healthcare.values()]
tech_dfs = [reset_index_if_needed(df) for df in tech.values()]

# Now concatenate all the prepared DataFrames
all_dfs = healthcare_dfs + tech_dfs  # Combine the lists
combined_df = pd.concat(all_dfs, ignore_index=True)

combined_df['Date'] = pd.to_datetime(combined_df['Date'])

In [31]:
# Group by 'Industry' and 'Date'
grouped = combined_df.groupby(['Industry', 'Date'])

# Apply mean only to numeric columns by filtering the DataFrame first
industry_means = grouped.apply(lambda x: x.select_dtypes(include=['number']).mean()).reset_index()

industry_means

C:\Users\emine\AppData\Local\Temp\ipykernel_12080\3467186542.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  industry_means = grouped.apply(lambda x: x.select_dtypes(include=['number']).mean()).reset_index()


,Industry,Date,close,volume,ttm_net_eps,pe_ratio,ttm_sales_per_share,book_value_per_share,ttm_fcf_per_share,current_assets,...,revenue,gross_profit,operating_income,ebitda,net_income,eps_earnings_per_share_diluted,ps_ratio,pb_ratio,pfcf_ratio,total_assets
0,Healthcare,2009-03-31,17.507322,2.078572e+07,3.609451,7.177199,14.328905,10.524829,3.021793,22.410000,...,7676.250650,5757.607349,2101.392642,2751.004675,2062.319532,0.911016,1.510941,1.738725,9.204826,74.850058
1,Healthcare,2009-06-30,17.813541,1.816895e+07,3.609451,7.177199,14.328905,10.524829,3.021793,22.410000,...,7676.250650,5757.607349,2101.392642,2751.004675,2062.319532,0.911016,1.510941,1.738725,9.204826,74.850058
2,Healthcare,2009-09-30,19.478132,1.401683e+07,3.602127,7.119063,14.485575,11.155017,3.060266,25.666250,...,7526.000000,5558.750000,2046.625000,2531.750000,1557.250000,0.705000,1.503612,1.736598,8.904741,74.778591
3,Healthcare,2009-12-31,21.289807,1.849621e+07,3.581424,7.200751,14.708025,11.737961,3.111963,26.426250,...,7726.750000,5717.250000,2303.625000,2777.000000,1937.625000,0.848750,1.499994,1.739013,8.660338,75.804136
4,Healthcare,2010-03-31,22.301888,7.541639e+06,3.545000,7.071212,14.982500,13.452203,3.102500,26.047500,...,9201.000000,6292.375000,1156.375000,1885.250000,2713.750000,1.285000,1.507371,1.744143,9.609553,75.688750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,Tech,2021-06-30,174.614973,2.741209e+07,4.702472,35.194926,20.188005,16.757256,8.140385,82.591587,...,31668.065760,17427.551020,10316.741497,11958.358277,9655.614512,1.554966,9.010210,14.987753,27.472252,185.198526
104,Tech,2021-09-30,197.527820,2.103329e+07,5.497344,34.239184,21.549286,16.937567,8.225826,86.319799,...,32310.491071,18307.808036,10665.243304,12401.006696,9585.910714,1.643571,9.575085,6.496561,35.350872,190.798214
105,Tech,2021-12-31,226.588497,2.552142e+07,5.856585,35.770783,22.169464,17.163929,8.242165,87.866183,...,32974.636161,18471.611607,10790.270089,12586.979911,9782.426339,1.682522,10.462204,-7.343298,36.511791,194.766272
106,Tech,2022-03-31,223.315234,2.517181e+07,6.548018,35.481386,23.836083,16.961152,7.557604,89.633779,...,41324.453917,22490.221198,13686.036866,15483.165899,11736.921659,1.883364,10.384480,11.178556,58.075119,200.032857


In [32]:
# Reset index if needed to ensure 'Date' and 'Industry' are columns
combined_df.reset_index(inplace=True, drop=True)
industry_means.reset_index(inplace=True, drop=True)

# Merge the industry average PE ratio into combined_df
# This assumes you have a column in industry_means named 'pe_ratio' representing the industry average
combined_df = pd.merge(combined_df, industry_means[['Industry', 'Date', 'pe_ratio']], on=['Industry', 'Date'], how='left', suffixes=('', '_industry_avg'))

combined_df

,Date,close,volume,ttm_net_eps,pe_ratio,ttm_sales_per_share,book_value_per_share,ttm_fcf_per_share,current_assets,current_ratio,...,ebitda,net_income,eps_earnings_per_share_diluted,ps_ratio,pb_ratio,pfcf_ratio,total_assets,Industry,Ticker,pe_ratio_industry_avg
0,2013-03-31,24.850064,9.420255e+06,2.546327,14.506392,11.378784,1.845384,3.860525,15.350000,2.270000,...,2130.000000,1540.000000,0.980000,3.161313,12.687595,10.720136,27.010000,Healthcare,ABBV,12.003430
1,2013-06-30,29.633117,7.263400e+06,2.545410,14.530981,11.436957,1.870000,3.832517,14.920000,2.210000,...,1532.000000,968.000000,0.600000,3.161289,12.566455,10.747170,27.170000,Healthcare,ABBV,14.680947
2,2013-09-30,30.314506,1.068848e+07,2.544990,14.593001,11.506570,2.240000,3.784147,15.660000,2.300000,...,1689.000000,1068.000000,0.660000,3.197231,12.587264,10.597877,27.910000,Healthcare,ABBV,15.223645
3,2013-12-31,33.915767,6.559198e+06,2.543723,14.547647,11.596222,2.250000,3.733563,16.540000,2.410000,...,1605.000000,964.000000,0.600000,3.122179,12.891034,10.248419,28.250000,Healthcare,ABBV,13.990938
4,2014-03-31,35.151819,2.920267e+06,2.560000,14.230469,11.700000,2.830000,3.600000,17.850000,2.600000,...,1735.000000,1128.000000,0.700000,3.113675,12.872792,10.119444,29.200000,Healthcare,ABBV,17.092982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
841,2021-06-30,76.380449,1.576937e+07,3.707302,18.207434,11.549365,2.907460,4.306032,47.871111,2.208889,...,4848.507937,4675.634921,1.564762,5.849987,24.914060,16.912413,122.649683,Tech,ORCL,35.194926
842,2021-09-30,86.482113,6.389084e+06,3.892969,20.614967,13.024531,1.289844,6.553125,52.797344,2.214687,...,4929.109375,3515.203125,1.189219,6.160260,-45.515981,12.244845,128.422656,Tech,ORCL,34.239184
843,2021-12-31,92.265293,6.864758e+06,3.441562,26.855310,13.533750,-1.513437,6.130000,41.606250,1.905938,...,2723.406250,1183.750000,0.406250,6.501125,-148.218592,15.063313,117.413125,Tech,ORCL,35.770783
844,2022-03-31,79.884973,9.064487e+06,2.534516,33.184367,14.276774,-3.419677,4.160645,31.302581,1.601774,...,1664.451613,75.870968,0.022258,5.894311,-24.525456,20.183533,107.545484,Tech,ORCL,35.481386


In [42]:
# Dropdown selection for tickers
ticker_dropdown = alt.binding_select(options=list(combined_df['Ticker'].unique()))
select_ticker = alt.selection_single(fields=['Ticker'], bind=ticker_dropdown, name="Select", value='ABBV')

# Base chart configuration for the selected ticker
base = alt.Chart(combined_df).transform_filter(
    select_ticker
).encode(
    x=alt.X('Date:T', title='Date')
).properties(
    width=800,
    height=400,
    title='Selected Ticker\'s PE vs. EPS'
)

# PE Ratio Line Chart
pe_ratio_chart = base.mark_line(color='blue').encode(
    y=alt.Y('pe_ratio:Q', title='PE Ratio (Blue)'),
    tooltip=['Ticker', 'Date:T', 'pe_ratio:Q']
)

# EPS Line Chart
eps_chart = base.mark_line(color='green').encode(
    y=alt.Y('eps_earnings_per_share_diluted:Q', title='EPS (Diluted) (Green)'),
    tooltip=['Ticker', 'Date:T', 'eps_earnings_per_share_diluted:Q']
)

# Combine the charts
combined_chart = alt.layer(pe_ratio_chart, eps_chart).resolve_scale(
    y='independent'
).add_selection(
    select_ticker
)

combined_chart.display()

c:\Users\emine\anaconda3\envs\ds\lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\emine\anaconda3\envs\ds\lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.LayerChart(...)

In [34]:
box_plot = alt.Chart(industry_means).mark_boxplot().encode(
    x='Industry:N',
    y='debt_to_equity_ratio:Q',
    color='Industry:N'
).properties(
    title='Distribution of Debt-to-Equity Ratios by Industry',
    width=200,
    height=200
)

box_plot.display()

alt.Chart(...)

In [40]:

# Dropdown selection for tickers, with 'ABBV' as the default selection
ticker_dropdown = alt.binding_select(options=list(combined_df['Ticker'].unique()))
select_ticker = alt.selection_single(fields=['Ticker'], bind=ticker_dropdown, name="Select", value = 'ABBV')

# Base chart configuration for the selected ticker
base = alt.Chart(combined_df).transform_filter(
    select_ticker  # Filter to show only the selected ticker
).encode(
    x=alt.X('yearmonth(Date):O', title='Date')  # Group by year-month for clarity
).properties(
    width=800,
    height=400,
    title="Selected Ticker's Gross Profit, Operating Income, and Closing Price Over Time"
)

# Side-by-side bar charts for 'gross_profit' and 'operating_income'
bar_chart = base.mark_bar().transform_fold(
    fold=['gross_profit', 'operating_income'],
    as_=['Metric', 'Value']
).encode(
    y=alt.Y('Value:Q', title='Financial Metrics'),
    color='Metric:N',
    tooltip=['Ticker', 'Date:T', 'Metric:N', 'Value:Q']
)

# Line chart for 'close' price overlaid on top
line_chart = base.mark_line(color='black', size=2).encode(
    y=alt.Y('close:Q', title='Closing Price'),
    tooltip=['Ticker', 'Date:T', 'close:Q']
)

# Text label displaying the industry of the selected ticker
industry_text = base.mark_text(align='left', dx=20, dy=-10).encode(
    text=alt.Text('Industry:N', title='Industry'),
    x=alt.value(10),  # position the text at the start of the x-axis
    y=alt.value(10)  # position the text near the top of the chart
)

# Combine the bar, line charts, and industry text
combined_chart = alt.layer(bar_chart, line_chart, industry_text).resolve_scale(
    y='independent'  # Allow independent scaling for the financial metrics and closing price
).add_selection(
    select_ticker
)

combined_chart.display()

c:\Users\emine\anaconda3\envs\ds\lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\emine\anaconda3\envs\ds\lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.LayerChart(...)

In [36]:
combined_df

,Date,close,volume,ttm_net_eps,pe_ratio,ttm_sales_per_share,book_value_per_share,ttm_fcf_per_share,current_assets,current_ratio,...,ebitda,net_income,eps_earnings_per_share_diluted,ps_ratio,pb_ratio,pfcf_ratio,total_assets,Industry,Ticker,pe_ratio_industry_avg
0,2013-03-31,24.850064,9.420255e+06,2.546327,14.506392,11.378784,1.845384,3.860525,15.350000,2.270000,...,2130.000000,1540.000000,0.980000,3.161313,12.687595,10.720136,27.010000,Healthcare,ABBV,12.003430
1,2013-06-30,29.633117,7.263400e+06,2.545410,14.530981,11.436957,1.870000,3.832517,14.920000,2.210000,...,1532.000000,968.000000,0.600000,3.161289,12.566455,10.747170,27.170000,Healthcare,ABBV,14.680947
2,2013-09-30,30.314506,1.068848e+07,2.544990,14.593001,11.506570,2.240000,3.784147,15.660000,2.300000,...,1689.000000,1068.000000,0.660000,3.197231,12.587264,10.597877,27.910000,Healthcare,ABBV,15.223645
3,2013-12-31,33.915767,6.559198e+06,2.543723,14.547647,11.596222,2.250000,3.733563,16.540000,2.410000,...,1605.000000,964.000000,0.600000,3.122179,12.891034,10.248419,28.250000,Healthcare,ABBV,13.990938
4,2014-03-31,35.151819,2.920267e+06,2.560000,14.230469,11.700000,2.830000,3.600000,17.850000,2.600000,...,1735.000000,1128.000000,0.700000,3.113675,12.872792,10.119444,29.200000,Healthcare,ABBV,17.092982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
841,2021-06-30,76.380449,1.576937e+07,3.707302,18.207434,11.549365,2.907460,4.306032,47.871111,2.208889,...,4848.507937,4675.634921,1.564762,5.849987,24.914060,16.912413,122.649683,Tech,ORCL,35.194926
842,2021-09-30,86.482113,6.389084e+06,3.892969,20.614967,13.024531,1.289844,6.553125,52.797344,2.214687,...,4929.109375,3515.203125,1.189219,6.160260,-45.515981,12.244845,128.422656,Tech,ORCL,34.239184
843,2021-12-31,92.265293,6.864758e+06,3.441562,26.855310,13.533750,-1.513437,6.130000,41.606250,1.905938,...,2723.406250,1183.750000,0.406250,6.501125,-148.218592,15.063313,117.413125,Tech,ORCL,35.770783
844,2022-03-31,79.884973,9.064487e+06,2.534516,33.184367,14.276774,-3.419677,4.160645,31.302581,1.601774,...,1664.451613,75.870968,0.022258,5.894311,-24.525456,20.183533,107.545484,Tech,ORCL,35.481386


In [44]:
# Dropdown selection for industries
industry_dropdown = alt.binding_select(options=combined_df['Industry'].unique().tolist())
select_industry = alt.selection_single(fields=['Industry'], bind=industry_dropdown, name="Select", value='Healthcare')

# Base chart for displaying data
base = alt.Chart(combined_df).transform_filter(
    select_industry  # Apply industry selection
).encode(
    x=alt.X('Date:T', title='Date'),  # Assume 'Date' is formatted as datetime
    y=alt.Y('close:Q', title='Closing Price'),
    color=alt.Color('Ticker:N', legend=alt.Legend(title="Ticker")),  # Color by ticker
    tooltip=['Ticker', 'Date:T', 'close:Q', 'Industry']
).properties(
    width=800,
    height=400,
    title="Price Over Time by Ticker within Selected Industry"
)

# Line chart for price over time
price_chart = base.mark_line().interactive()

# Display the chart
price_chart.add_selection(
    select_industry
).display()

c:\Users\emine\anaconda3\envs\ds\lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\emine\anaconda3\envs\ds\lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.Chart(...)